# Explore the preprocessed data

In [ ]:
import os
import json

import pandas as pd
import torch
import torchaudio
from matplotlib import pyplot as plt

## Config

In [ ]:
annotations_path = "../dataset/preprocessed_data/frame-2048_hop-1024_chunk-128_spec-mel_mels-128_silence-label/labels.tsv"
label2idx_path = "../dataset/preprocessed_data/frame-2048_hop-1024_chunk-128_spec-mel_mels-128_silence-label/label2idx.json"
audio_dir = "../dataset/preprocessed_data/frame-2048_hop-1024_chunk-128_spec-mel_mels-128_silence-label/audio_tensors"

In [ ]:
annotations = pd.read_csv(annotations_path, sep="\t")
annotations.head()

In [ ]:
with open(label2idx_path, "r") as file_handle:
    label2idx = json.load(file_handle)

labels = list(label2idx)
print(label2idx) 
print(labels)

## Check labels distributions

In [ ]:
display(annotations[labels].sum())
total_annot = len(annotations)
total_labels = annotations[labels].sum().sum()
print(f"{total_labels=}")
print(f"{total_annot=}")
print(f"{(total_labels/total_annot)=}")

## Check extracted spectrograms

In [ ]:
def plot_spectrogram(specgram, title=None, ylabel="freq_bin", interpolation="antialiased"):
    plt.figure(figsize=(30, 10))
    plt.title(title or "Spectrogram (db)")
    plt.ylabel(ylabel)
    plt.xlabel("frame")
    amplitude_2_DB = torchaudio.transforms.AmplitudeToDB()
    plt.imshow(amplitude_2_DB(specgram), origin="lower", aspect="auto", interpolation=interpolation)
    plt.colorbar()
    plt.show()

def plot_mask(mask, labels):
    plt.figure(figsize=(30, 10))
    plt.imshow(mask, aspect="auto", interpolation="none", cmap="jet")
    plt.yticks(range(len(labels)), labels=labels)
    plt.xlabel("Frame")
    plt.colorbar()
    plt.show()

In [ ]:
mask = annotations.whistle == 1
#mask &= annotations.click == 1
#mask &= annotations.cetaceans_allfreq == 1
selected_annot = annotations[mask]
selected_annot.head()

In [ ]:
sample_annot = selected_annot.iloc[16]
feature_data = torch.load(os.path.join(audio_dir, sample_annot.feature_path))
mask_data = torch.load(os.path.join(audio_dir, sample_annot.mask_path))
print(f"{feature_data.shape=}")
print(f"{mask_data.shape=}")

In [ ]:
plot_spectrogram(torch.from_numpy(feature_data)[0])
plot_mask(torch.from_numpy(mask_data), labels)